# Start simple, Start linearly

The program below will mark the first RL algorithm truly implemented in this project. As such, to see if I've got the basics down, I started with a simply linear policy to learn. The algorithm contains two main parts. 

* The first is to learn a surrogate dynamics model of the environment. I personally like to think of this as the agent's belief of how the environment behaves, so the term surrogate doesn't too appropriate. A GP model will be used, with a gaussian covariance kernel. 

* The second is to use this belief of the system to learn an optimal policy. As mentioned previously, a linear policy structure will be used. Furthermore, bayesian optimisation with expected improvement will be used to find the optimal policy values. 

Let's see how this goes. Fingers crossed. 

In [1]:
import gym
import GPy
import GPyOpt
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import gympy

Define the characteristics of the particular exercise. In this case, for example, the type of learning analysis will be 'Reset' since the initial state of the environment is reset for each test point. 

In [2]:
analysisType_learning = 'Reset'

Set up the gym environment and create the array definitions for  input, output and predictions. Also generate initial values for the relevant variables. 

In [3]:
env, observations, inputsNumpy, observationsNumpy, rewardsNumpy, action, predictionPDF = gympy.setupEnvironment(defaultEnvironment = 'Pendulum-v0', whichVersion = 0)
k_dynamics = GPy.kern.RBF(input_dim=4, variance=1., lengthscale=1.)
k_rewards = GPy.kern.RBF(input_dim=3, variance=1., lengthscale=1.)

Please enter the environment: 


[2016-11-15 22:03:59,248] Making new env: Pendulum-v0


Initial learning phase - learn surrogate model. This will be done with 500 random training data points. As such a loop will first be initiated to generate the training points.

In [60]:
env.reset(whichVersion = 1)
for attempt in range(100):
    [inputsNumpy, bufferInput, actionToTake] = gympy.appendInputArray(inputsNumpy, action, observations, attempt)
    [observations, rewards, done, info] = env.step(actionToTake)
    [observationsNumpy, bufferObservations] = gympy.appendObservationsArray(observationsNumpy, observations, attempt)
    [rewardsNumpy, bufferRewards] = gympy.appendRewardsArray(rewardsNumpy, rewards, attempt)
#     observations = env.reset()
    env.render()

AttributeError: 'numpy.ndarray' object has no attribute 'sample'

Use the training points to generate the training points. 

In [6]:
m_dynamics = gympy.generateModel(inputsNumpy, observationsNumpy, k_dynamics)
m_dynamics.optimize()

[2016-11-15 22:04:12,951] initializing Y
[2016-11-15 22:04:12,952] initializing inference method
[2016-11-15 22:04:12,953] adding kernel and likelihood as parameters


Once the learning phase has been completed, it is necessary to define the goal state (that is the terminal state) and how the reward can be calculated in terms of the current state with respect to the goal state. 

In this case, the reward is calculated as the sum of the geometric distances between the each of the current states and the goal states. Please note that since each of these will be $ \geqslant \: 0$, it is technically speaking a cost, rather than a reward. However, since the difference betweeen the two is merely a negative symbol, reward and cost will be used interchangebly. 

In [7]:
goalState = np.array([1,0,0])

In [8]:
def generateTotalReward(predictionPDF, goalState, totalReward):
    predictionPDF[0,2] = predictionPDF[0,1]/8
    if abs(np.arccos(predictionPDF[0,0])) > 1*np.pi/180:
        multiplier = 100
    else:
        multiplier = 1
    currentReward = multiplier*(np.sqrt(np.sum((predictionPDF-goalState)*(predictionPDF-goalState))))
    totalReward = totalReward + currentReward
    
    return totalReward

The policy must now be defined in terms of its parameters. In this particular example, a linear policy was chosen, for simplicity. This will be of the form:

$$ \pi(s_t) = \phi(s_t)^T \mathbf{\theta},$$

where the basis functions in $\phi$ are simply the state variables, and the vector $\theta$ represents the weights. Thus the action will be selected by policy that looks like:

$$ a_t = s_{t,1}\theta_1 + s_{t,2}\theta_2  +  s_{t,3}\theta_3 + \theta_4, $$

since this particular problem has 3 state variables. The reinforcement learning problem now becomes one of finiding the values of $\theta$ such that the cost defined earlier is minimised. 

The first step of this is to generate random initial values for the parameter values.

In [9]:
policyParameters =  np.random.uniform(-1., 1., ([1, 4]))

Now, a function is defined to return the action that needs to be taken as given by the policy defined by the current parameter values.

In [10]:
def action_fromPolicy(policyParameters, bufferObservations_policy):
    actionToTake_policy = 0
    [r,c] = policyParameters.shape
    for i in range(c):
        if i < c-1:
            actionToTake_policy = actionToTake_policy + (bufferObservations_policy[0,i] * policyParameters[0,i])
        else:
            actionToTake_policy = actionToTake_policy + policyParameters[0,i]
                

    return actionToTake_policy

Finding the policy parameters that minimise the total reward is clearly an optimisation problem. As such the optimisation method that will be used for this particular notebook is Bayesian Optimisation using Expected Improvement. Simply put, this methods chooses the next test point based on the highest expected improvement from the current minimun point (for a minimising problem that is). Thusly, a good mix of exploration and exploitation can be utilised to find the global minimum given the state box.

The python library GPyOpt will be utilised for this. This requires the definition of an objective function; this is function that is to be optimised. In the present work, this will be the mapping from the policyy parameters to the total reward. Since this is a model based RL algorithm, the GP model that was trained previously will be used to generate the total reward. 

In [27]:
def objectiveFunction(policyParameters):
    thetaBound = np.array([np.pi*5*180,1])
    
    thetaObservations = np.random.uniform(low=-thetaBound, high=thetaBound)

    bufferObservations_policy = np.array([np.cos(thetaObservations[0]), np.sin(thetaObservations[0]), thetaObservations[1]])

    bufferObservations_policy = np.reshape(bufferObservations_policy, ([1,3]))
    bufferInpust_policy = np.zeros((1,4))
    totalIterations = 100
    totalReward = 0
    
    goalState = np.array([-1,0,0])
    
#     env.setState(np.arccos(bufferObservations_policy[0,0]), bufferObservations_policy[0,2])
#     env.render()
    
    actionToTake_policy = action_fromPolicy(policyParameters, bufferObservations_policy)
    
    for i in range(totalIterations):
        
#         if abs(actionToTake_policy) > 2:
#             totalReward = totalReward + 1000
        bufferActionToTake = np.array([actionToTake_policy])
        bufferActionToTake = np.reshape(bufferActionToTake, ([1,1]))
        bufferInputs_policy = np.append(bufferObservations_policy, bufferActionToTake, axis = 1) 

        predictionPDF = m_dynamics.predict(bufferInputs_policy)[0]
                
        totalReward = generateTotalReward(predictionPDF, goalState, totalReward)
        
        actionToTake_policy = action_fromPolicy(policyParameters, predictionPDF)
#         env.setState(np.arccos(predictionPDF[0,0]), predictionPDF[0,2])
#         env.render()
        
    return totalReward

In [28]:
bounds = [{'domain': (-1,1), 'name': 'var_1', 'type': 'continuous', 'dimensionality':3},
         {'domain': (-10,10), 'name': 'var_4', 'type': 'continuous', 'dimensionality':1}]

In [31]:
myBopt = GPyOpt.methods.BayesianOptimization(f = objectiveFunction, domain = bounds, acquisition_type ='EI')

[2016-11-15 22:19:46,645] initializing Y
[2016-11-15 22:19:46,646] initializing inference method
[2016-11-15 22:19:46,646] adding kernel and likelihood as parameters


In [54]:
max_iter = 10              # evaluation budget
myBopt.run_optimization(max_iter)   # run optimization
print myBopt.Y.shape


(224, 1)


In [58]:
policyParameters_optimal = myBopt.X[np.argmin(myBopt.Y)]
print min(myBopt.Y)
policyParameters_optimal = np.reshape(policyParameters_optimal, ([1,4]))
print policyParameters_optimal

[ 6.14298203]
[[ 1.          1.         -1.         -8.44482693]]


In [69]:
env = gym.make('Pendulum-v0')

policyParameters_optimal = np.array([[-10,-10,-10,0]])
env.reset(whichVersion = 1)
# observations = env.state(inputTheta = initialStates[0,0], inputThetaDot = initialStates[0.1])
for i in range (500):
    env.render()
    observations = np.reshape(observations, ([1,3]))
    action = action_fromPolicy(policyParameters_optimal, observations)
    action = np.array([action])
    print action
    observations, rewards, done, info = env.step(action)

[2016-11-15 23:53:44,013] Making new env: Pendulum-v0


[ 3.5403856]
[-13.21918359]
[-10.3710819]
[-7.38893283]
[-4.15501112]
[-0.53542592]
[ 1.30610032]
[ 0.72138894]
[ 1.37140239]
[ 1.36665221]
[ 1.68370724]
[ 1.81655195]
[ 2.02790888]
[ 2.22609111]
[ 2.68798451]
[ 3.42482487]
[ 4.45695695]
[ 5.81358207]
[ 7.53246246]
[ 9.65929243]
[ 12.24629876]
[ 15.34940602]
[ 19.02298597]
[ 23.31083307]
[ 28.23169208]
[ 33.75773355]
[ 39.78547498]
[ 46.10176893]
[ 52.35349777]
[ 58.03784142]
[ 62.53577808]
[ 65.2053798]
[ 65.52525101]
[ 63.23954681]
[ 58.43477543]
[ 51.50496188]
[ 43.02473201]
[ 33.59829767]
[ 23.74967093]
[ 13.88006896]
[ 4.28071496]
[-4.82639898]
[-7.66465669]
[-10.16924839]
[-12.27568659]
[-13.9208953]
[-15.04621968]
[-15.60183651]
[-15.55166296]
[-14.87768847]
[-13.58281661]
[-11.69178997]
[-9.25039534]
[-6.32363061]
[-2.99365432]
[ 0.64188389]
[ 0.68255145]
[ 1.0265151]
[ 1.22246087]
[ 1.49006657]
[ 1.71901189]
[ 1.96172056]
[ 2.19016603]
[ 2.69603474]
[ 3.5141184]
[ 4.61540249]
[ 5.96577663]
[ 7.52694635]
[ 9.25693589]
[ 11.1102

ArgumentError: argument 2: <type 'exceptions.TypeError'>: wrong type

In [ ]:
for iterations in range(100)
    

In [62]:
for x in range(100):

SyntaxError: unexpected EOF while parsing (<ipython-input-62-6340df6f05f3>, line 1)